In [ ]:
from queries import start_elastic_search, index_documents



In [ ]:
es = start_elastic_search()
collection_file_name = "data01/FIR-s05-medline.json"
index_name = "genomics-base"

# Index the documents
index_documents(es, collection_file_name, index_name)